Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install tensorflow
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=2eaf9bbcbe2a05958ddfb460c66497c3e9d2101186914ecab3c5a49f6f0e54d5
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
import os
import pathlib
import numpy as np
import pandas as pd
import scipy.stats
import tensorflow as tf
import joblib
from tqdm.notebook import tqdm

'''
#################### MDO ###########################

import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F
'''

'\n#################### MDO ###########################\n\nimport torch\nfrom torch.nn import Linear\nfrom torch.nn import Sequential\nfrom torch.functional import F\n'

In [ ]:
NUMERAI_S3_BUCKET_URL = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com"

EXAMPLE_PREDS_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_example_predictions_data.csv.xz"
TOURNAMENT_DATA_URL = NUMERAI_S3_BUCKET_URL + "/latest_numerai_tournament_data.csv.xz"
TEST_DATA = NUMERAI_S3_BUCKET_URL + "/latest_numerai_max_test_era_data.csv.xz"

###IMPORTANT! DELETE THE FILE BELOW IF YOU CHANGE MODELS! OTHERWISE, RENAME THE FILE FOR YOUR VARIOUS MODELS###
LM_CACHE_FILE = pathlib.Path("neutralization.cache.joblib")

In [ ]:

@tf.function(experimental_relax_shapes=True, experimental_compile=True)
def exposures(x, y):
    x = x - tf.math.reduce_mean(x, axis=0)
    x = x / tf.norm(x, axis=0)
    y = y - tf.math.reduce_mean(y, axis=0)
    y = y / tf.norm(y, axis=0)
    return tf.matmul(x, y, transpose_a=True)

#################### MDO ###########################
'''
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)
'''

'\ndef exposures(x, y):\n    x = x - x.mean(dim=0)\n    x = x / x.norm(dim=0)\n    y = y - y.mean(dim=0)\n    y = y / y.norm(dim=0)\n    return torch.matmul(x.T, y)\n'

In [ ]:
@tf.function(experimental_relax_shapes=True)
def train_loop_body(model, feats, pred, target_exps):
    with tf.GradientTape() as tape:
        exps = exposures(feats, pred[:, None] - model(feats, training=True))
        loss = tf.reduce_sum(tf.nn.relu(tf.nn.relu(exps) - tf.nn.relu(target_exps)) +
                             tf.nn.relu(tf.nn.relu(-exps) - tf.nn.relu(-target_exps)))
    return loss, tape.gradient(loss, model.trainable_variables)

In [ ]:
def train_loop(model, optimizer, feats, pred, target_exps, era):
    for i in range(1000000):
        loss, grads = train_loop_body(model, feats, pred, target_exps)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        if loss < 1e-7:
            break
        if i % 10000 == 0:
            tqdm.write(f'era: {era[3:]} loss: {loss:0.7f}', end='\r')

In [ ]:
def reduce_exposure(prediction, features, max_exp, era, weights=None):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(310),
        tf.keras.experimental.LinearModel(use_bias=False),
    ])
    feats = tf.convert_to_tensor(features - 0.5, dtype=tf.float32)
    pred = tf.convert_to_tensor(prediction, dtype=tf.float32)
    if weights is None:
        optimizer = tf.keras.optimizers.Adamax()
        start_exp = exposures(feats, pred[:, None])
        target_exps = tf.clip_by_value(start_exp, -max_exp, max_exp)
        train_loop(model, optimizer, feats, pred, target_exps, era)
    else:
        model.set_weights(weights)
    return pred[:,None] - model(feats), model.get_weights()


###################   MDO    ########################################

'''
def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer
'''

"\ndef reduce_exposure(prediction, features, max_exp):\n    # linear model of features that will be used to partially neutralize predictions\n    lin = Linear(features.shape[1],  1, bias=False)\n    lin.weight.data.fill_(0.)\n    model = Sequential(lin)\n    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)\n    feats = torch.tensor(np.float32(features)-.5)\n    pred = torch.tensor(np.float32(prediction))\n    start_exp = exposures(feats, pred[:,None])\n    # set target exposure for each feature to be <= current exposure\n    # if current exposure is less than max_exp, or <= max_exp if  \n    # current exposure is > max_exp\n    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)\n\n    for i in range(100000):\n        optimizer.zero_grad()\n        # calculate feature exposures of current linear neutralization\n        exps = exposures(feats, pred[:,None]-model(feats))\n        # loss is positive when any exposures exceed their target\n        loss = (F.relu(F.relu(exps)-F.

In [ ]:
def reduce_all_exposures(df, column=["prediction"], neutralizers=None,
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1): ###<-----SELECT YOUR MAXIMUM FEATURE EXPOSURE HERE###
    if neutralizers is None:
        neutralizers = [x for x in df.columns if x.startswith("feature")]
    neutralized = []
    if LM_CACHE_FILE.is_file():
        cache = joblib.load(LM_CACHE_FILE)
        # Remove weights for eraX if we'd accidentally saved it in the past.
        cache.pop("eraX", None)
    else:
        cache = {}
    for era in tqdm(df[era_col].unique()):
        tqdm.write(era, end='\r')
        df_era = df[df[era_col] == era]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values

        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, weights = reduce_exposure(scores, exposure_values,
                                          max_exp, era, cache.get(era))
        if era not in cache and era != "eraX":
            cache[era] = weights
            joblib.dump(cache, LM_CACHE_FILE)
        scores /= tf.math.reduce_std(scores)
        scores -= tf.reduce_min(scores)
        scores /= tf.reduce_max(scores)
        neutralized.append(scores.numpy())

    predictions = pd.DataFrame(np.concatenate(neutralized),
                               columns=column, index=df.index)
    return predictions


############################   MDO   ######################################
'''
def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)
'''

'\ndef reduce_all_exposures(df, column, neutralizers=[],\n                                     normalize=True,\n                                     gaussianize=True,\n                                     era_col="era",\n                                     max_exp=0.1):\n    unique_eras = df[era_col].unique()\n    computed = []\n    for u in unique_eras:\n        print(u, \'\r\')\n        df_era = df[df[era_col] == u]\n        scores = df_era[column].values\n        exposure_values = df_era[neutralizers].values\n        \n        if normalize:\n            scores2 = []\n            for x in scores.T:\n                x = (scipy.stats.rankdata(x, method=\'ordinal\') - .5) / len(x)\n                if gaussianize:\n                    x = scipy.stats.norm.ppf(x)\n                scores2.append(x)\n            scores = np.array(scores2)[0]\n\n        scores, neut = reduce_exposure(scores, exposure_values, max_exp)\n\n        scores /= scores.std()\n\n        computed.append(scores.detach

In [ ]:
#If CUDA isn't set up properly for Tensorflow, then at least maximize the number of threads available for CPU
if not tf.config.list_physical_devices('GPU'):  # No GPU(s) found
    tf.config.threading.set_inter_op_parallelism_threads(2)
    tf.config.threading.set_intra_op_parallelism_threads(os.cpu_count() // 2)

In [ ]:
#read-in or download the example predictions
exp_df = pd.read_csv(EXAMPLE_PREDS_URL, index_col=0)

exp_df.head()

,prediction
id,
n0003aa52cab36c2,0.49
n000920ed083903f,0.49
n0038e640522c4a6,0.53
n004ac94a87dc54b,0.51
n0052fe97ea0c05f,0.50


In [ ]:
exp_df.head()

,prediction
id,
n0003aa52cab36c2,0.49
n000920ed083903f,0.49
n0038e640522c4a6,0.53
n004ac94a87dc54b,0.51
n0052fe97ea0c05f,0.50


In [ ]:
#download the tournament data
tournament_df = pd.read_csv(TOURNAMENT_DATA_URL, index_col=0)
tournament_df.head()

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,0.25,0.50,0.5,0.25,0.0,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.0,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.0,0.25,0.5,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.5,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,0.75,0.25,0.0,0.75,0.5,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.0,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.5,0.25,1.0,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.5,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,1.00,0.50,0.5,1.00,1.0,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.5,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.25,0.5,0.00,0.0,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.0,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,0.50,0.75,1.0,0.75,0.0,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.0,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,0.25,0.25,0.5,0.50,1.0,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.0,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.00,0.5,0.50,0.0,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.0,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [ ]:

tournament_df

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.25,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.00,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffb0e4e6

In [ ]:
#merge them together
full_df = pd.merge(tournament_df, exp_df, left_index=True, right_index=True)

In [ ]:
#this cell executes the full script above and neutralizes the predictions to achieve a maximum 0.1 Feature Exposure
neutralized_df = reduce_all_exposures(full_df)

In [ ]:
neutralized_df

,prediction
id,
n0003aa52cab36c2,0.320204
n000920ed083903f,0.308392
n0038e640522c4a6,0.575278
n004ac94a87dc54b,0.556190
n0052fe97ea0c05f,0.409397
...,...
nffb0e4e60b166ea,0.540612
nffc2bf82af58f1c,0.623061
nffea596c525e547,0.686991


In [ ]:
#save your prediction file locally
neutralized_df.to_csv("FisaGol_1.csv")

In [ ]:
!cp FisaGol_1.csv "drive/My Drive/PHOENIXSIGMA/"

In [ ]:
########################### MDO CODE ######################

In [ ]:
########################### MDO CODE ######################

In [ ]:
########################### MDO CODE ######################

In [ ]:
########################### MDO CODE ######################

In [ ]:
import numpy as np 
import pandas as pd 
from xgboost import XGBRegressor 
import torch
from torch.autograd import grad


trainval=pd.read_parquet("numerai_training_validation_target_nomi.parquet")
train = trainval[trainval.data_type=='train']

target = "target_nomi" 
feature_columns = [c for c in trainval if c.startswith("feature")] 

# fit an initial model
model_init = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, nthread=6)
model_init.fit(train[feature_columns], train[target])

# get prediction from initial model as starting point to improve upon
base_margin = model_init.predict(train[feature_columns])

# get indexes for each era
era_idx = [np.where(train.era==uera)[0] for uera in train.era.unique()]


# define adjusted sharpe in terms of cost adjusted numerai sharpe
def numerai_sharpe(x):
    return (x.mean() -0.010415154) / x.std()

def skew(x):
    mx = x.mean()
    m2 = ((x-mx)**2).mean()
    m3 = ((x-mx)**3).mean()
    return m3/(m2**1.5)    

def kurtosis(x):
    mx = x.mean()
    m4 = ((x-mx)**4).mean()
    m2 = ((x-mx)**2).mean()
    return (m4/(m2**2))-3

def adj_sharpe(x):
    return numerai_sharpe(x) * (1 + ((skew(x) / 6) * numerai_sharpe(x)) - ((kurtosis(x) / 24) * (numerai_sharpe(x) ** 2)))

# use correlation as the measure of fit
def corr(pred, target):
    pred_n = pred - pred.mean(dim=0)
    pred_n = pred_n / pred_n.norm(dim=0)

    target_n = target - target.mean(dim=0)
    target_n = target_n / target_n.norm(dim=0)
    l = torch.matmul(pred_n.T, target_n)
    return l

# definte a custom objective for XGBoost
def adj_sharpe_obj(ytrue, ypred):
    # convert to pytorch tensors
    ypred_th = torch.tensor(ypred, requires_grad=True)
    ytrue_th = torch.tensor(ytrue)
    all_corrs = []

    # get correlations in each era
    for ee in era_idx:
        score = corr(ypred_th[ee], ytrue_th[ee])
        all_corrs.append(score)

    all_corrs = torch.stack(all_corrs)

    # calculate adjusted sharpe using correlations
    loss = -adj_sharpe(all_corrs)
    print(f'Current loss:{loss}')

    # calculate gradient and convert to numpy
    loss_grads = grad(loss, ypred_th, create_graph=True)[0]
    loss_grads = loss_grads.detach().numpy()

    # return gradient and ones instead of Hessian diagonal
    return loss_grads, np.ones(loss_grads.shape)


model_adj_sharpe = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=200, nthread=6, colsample_bytree=0.1, objective=adj_sharpe_obj)
model_adj_sharpe.fit(train[feature_columns], train[target], base_margin=base_margin)


In [ ]:
def neutralize(df, target="prediction_kazutsugi", by=None, proportion=1.0):
    if by is None:
        by = [x for x in df.columns if x.startswith('feature')]

    scores = df[target]
    exposures = df[by].values

    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))

    scores -= proportion * (exposures @ (np.linalg.pinv(exposures) @ scores.values))
    return scores / scores.std()

In [ ]:
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)


TOURNAMENT_NAME = "kazutsugi"
PREDICTION_NAME = f"prediction_{TOURNAMENT_NAME}"

## Get output of your model
# data[PREDICTION_NAME] = model.predict(data[feature_names])

# reduce feature exposure in each era to max_exp
data_rfe_10 = reduce_all_exposures(data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.10)

# replace prediction with reduced feature exposure prediction and rescale to [0,1]
data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
data[PREDICTION_NAME] -= data[PREDICTION_NAME].min()
data[PREDICTION_NAME] /= data[PREDICTION_NAME].max()

In [ ]:
def pearson_cumsom_loss(y_true, y_pred):
    '''
    optmize negative pearson coefficient loss
    :param y_true:
    :param y_pred:
    :return:
    '''
    if isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    n = len(y_true)
    y_bar = y_true.mean()
    yhat_bar = y_pred.mean()
    c = 1 / ((y_true - y_bar) ** 2).sum().sqrt()  # constant variable
    b = ((y_pred - yhat_bar) ** 2).sum().sqrt()  # std of pred

    a_i = y_true - y_bar
    d_i = y_pred - yhat_bar
    a = (a_i * d_i).sum()
    gradient = c * (a_i / b - a * d_i / b**3)
    hessian = - (np.matmul(a_i.reshape(-1, 1), d_i.reshape(1, -1)) + np.matmul(d_i.reshape(-1, 1), a_i.reshape(1, -1))) / b ** 3 + \
              3 * a * np.matmul(d_i.reshape(-1, 1), d_i.reshape(1, -1)) / b**5 + a/(n*b**3)
    hessian = hessian - np.ones(shape=(n, n)) * a/b**3
    hessian *= c
    return -gradient, -hessian

In [ ]:
train_df = ...
era_idx = [np.where(eras == e)[0] for e in np.unique(train_df.eras)]

def loss_fn(target, pred):
    pred = torch.tensor(pred, requires_grad=True)
    target = torch.tensor(target)
    corrs = torch.stack([spearman(target[e], pred[e]) for e in era_idx])
    sharpe = adj_sharpe(corrs)
    gradient = torch.autograd.grad(sharpe, pred)[0].detach().numpy()
    hessian = np.ones_like(gradient)  # ones for hessian should be ~okay~
    return gradient, hessian

model = XGBRegressor(objective=loss_fn)

In [ ]:
from fast_soft_sort.tf_ops import soft_rank
import tensorflow as tf
import numpy as np

def pearson_corr(x, y):
    
    xy_t = tf.concat([x, y], axis=0)
    mean_t = tf.reduce_mean(xy_t, axis=1, keepdims=True)
    cov_t = ((xy_t-mean_t) @ tf.transpose(xy_t-mean_t))/(x.shape[1]-1)
    cov2_t = tf.linalg.diag(1/tf.sqrt(tf.linalg.diag_part(cov_t)))
    corr_matrix = cov2_t @ cov_t @ cov2_t
    corr = tf.reduce_mean(corr_matrix) * 2 - 1 # equivalent to taking element [0][1] assuming the 2x2 corr matrix is symmetric and the diagonals are 1
    
    return corr

def spearman_corr(x, y):
    
    ranks = soft_rank(x, regularization_strength=0.1)
    corr = pearson_corr(ranks, y)
    
    return corr

def get_value_grad_and_hess(x, y, f):
    
    x_var = tf.Variable(x, dtype=tf.float32)
    y_var = tf.Variable(y, dtype=tf.float32)
        
    val, grad, hess = None, None, None

    with tf.GradientTape() as t2:
    
        with tf.GradientTape() as t1:
            
            val = f(x_var, y_var)
        
        grad = t1.gradient(val, x_var)    

    hess = t2.jacobian(grad, x_var)

    return val, grad, hess

# test with random input
x = np.random.rand(1, 10) # predictions
y = np.random.rand(1, 10) # labels

print('pearson:')
val, grad, hess = get_value_grad_and_hess(x, y, pearson_corr)
print(' value:',  val)
print(' gradient:', grad)
print(' hessian:', hess)

print('spearman:')
val, grad, hess = get_value_grad_and_hess(x, y, spearman_corr)
print(' value:',  val)
print(' gradient:', grad)
print(' hessian:', hess)


In [ ]:
https://github.com/iRyanBell/numerai_kfold_ensemble/blob/master/NumerAI_KFold_Ensemble.ipynb

In [ ]:
https://github.com/iRyanBell/ARIMA_Stock_Correlations/blob/master/ARIMA_Stock_Correlations.ipynb

In [ ]:
import numpy as np
from scipy.stats import spearmanr

def loss_fn(target, pred):
    pred = torch.tensor(pred, requires_grad=True)
    target = torch.tensor(target)
    corrs = torch.stack([spearmanr(target[e], pred[e]) for e in era_idx])
    sharpe = adj_sharpe(corrs)
    gradient = torch.autograd.grad(sharpe, pred)[0].detach().numpy()
    hessian = np.ones_like(gradient)  # ones for hessian should be ~okay~
    return gradient, hessian

#model = XGBRegressor(objective=loss_fn)

In [ ]:
import numpy as np 
import pandas as pd 
from xgboost import XGBRegressor 
import torch
from torch.autograd import grad

# fit an initial model
model_init = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, nthread=6)
model_init.fit(training_data[feature_names], training_data[TARGET_NAME])

# get prediction from initial model as starting point to improve upon
base_margin = model_init.predict(training_data[feature_names])

# get indexes for each era
era_idx = [np.where(training_data.era==uera)[0] for uera in training_data.era.unique()]